# pyVolcano
Volcano plot working over matplotlib, numpy and pandas

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
pval_col = 'padj'
gene_col = 'row'
log_col = 'log2FoldChange'
pval_thresh = 0.01
log_thresh = 1
up_color = 'green'
down_color = 'red'
title = 'No_cell_comp'
n_names2show = 10

In [3]:
in_file = '../data/DE_data.xlsx'

In [6]:
file_type = in_file.split('.')[-1]
if file_type == 'tsv':
    DF = pd.read_csv(in_file,sep='\t',index_col=0).reset_index(drop=True)
elif file_type == 'xlsx':
    DF = pd.read_excel(in_file,index_col=0)
elif file_type == 'csv':
    DF = pd.read_csv(in_file,sep=',',index_col=0).reset_index(drop=True)
else:
    raise NameError("Invalid input format. Has to be either .tsv, .csv or .xlsx.")

In [7]:
DF = DF.loc[DF['design']=='No_cell_comp']
DF

,row,log2FoldChange,padj,design,DE
1,ENSG00000278267,-0.175689,0.701615,No_cell_comp,NO
2,ENSG00000238009,-0.331684,0.481832,No_cell_comp,NO
3,ENSG00000239906,-0.338826,0.587608,No_cell_comp,NO
4,ENSG00000241860,-0.380158,0.188621,No_cell_comp,NO
5,ENSG00000222623,0.121562,0.938411,No_cell_comp,NO
...,...,...,...,...,...
28355,ENSG00000198695,1.757460,0.000385,No_cell_comp,UP
28356,ENSG00000210194,3.145639,0.001141,No_cell_comp,UP
28357,ENSG00000198727,0.005482,0.994661,No_cell_comp,NO
28358,ENSG00000210195,-0.273726,0.714065,No_cell_comp,NO


## Sort DF properly

In [ ]:
DF.insert(2,'absLogF',np.absolute(DF.loc[:,'log2FoldChange']))
DF

In [ ]:
DF = DF.sort_values(['padj','log2FoldChange'],ascending=[True,False]).reset_index(drop=True)
DF

## Insert color

In [ ]:
DF.insert(4,'color','black')
DF

In [ ]:
down = (DF.loc[:,pval_col]<pval_thresh)&(DF.loc[:,log_col]<-log_thresh)
up = (DF.loc[:,pval_col]<pval_thresh)&(DF.loc[:,log_col]>log_thresh)

In [ ]:
DF.loc[down,'color'] = down_color
DF.loc[up,'color'] = up_color

In [ ]:
DF

## Plot

In [ ]:
x = DF.loc[:,log_col].values
y = -np.log10(DF.loc[:,pval_col].values)
names = DF.loc[:,gene_col]

fig,ax = plt.subplots(figsize=[8,8])
ax.axhline(-np.log10(pval_thresh),color='gray',linestyle='--')
ax.axvline(log_thresh,color='gray',linestyle='--')
ax.axvline(-log_thresh,color='gray',linestyle='--')
ax.scatter(x,y,c=DF.loc[:,'color'].values,s=3)
for i in range(n_names2show):
    ha = "right" if x[i] > 0 else "left"
    ax.text(x[i], y[i] , s = names[i],ha=ha)
ax.set_ylabel(r'$-log_{10}(pval)$')
ax.set_xlabel(r'$log_2FoldChange$')
ax.set_title(title)